In [298]:
import pandas as pd
import numpy as np
from jiwer import wer, cer, mer
from string import digits

In [394]:
df = pd.read_csv('text_evaluation_data.csv', sep=";")
df.head()

,DETECTED_TEXT,GROUND_TRUTH
0,1.12,!?
1,A thousand curses. prisoner is gone.,A thousand curses! Prisoner is gone! ...
2,My poor snowy!. Did those beasts flatten him?.,My poor Snowy!… Did those beasts flatten him?
3,here we are. Snowy!,Here we are Snowy! ... Chicago!
4,Beaten! He's defeated again! me unless,Beaten! He's defeated me again! ... Unless ...


In [395]:
df.loc[pd.isna(df['DETECTED_TEXT']) == 1, "DETECTED_TEXT"] = ""

In [396]:
wer_values = []
cer_values = []
counter_0 = 0
counter_01 = 0
counter_02 = 0
counter_05 = 0
counter_worst = 0

for i in range(len(df)):
    hypo = df["DETECTED_TEXT"][i]
    ref = df["GROUND_TRUTH"][i]
    
    cer_i = np.where(cer(ref, hypo) > 1, 1, cer(ref, hypo))
    wer_i = np.where(wer(ref, hypo) > 1, 1, wer(ref, hypo))
    
    if wer_i == 0:
        counter_0 += 1
    elif wer_i < 0.1:
        counter_01 += 1
    elif wer_i < 0.2:
        counter_02 += 1
    elif wer_i < 0.5:
        counter_05 += 1
    else:
        counter_worst += 1
    
    #print(i,"WER", wer_i)
    #print(i, "CER", cer_i)
    
    wer_values.append(wer_i)
    cer_values.append(cer_i)

In [397]:
print(counter_0, counter_01, counter_02, counter_05, counter_worst)

14 5 19 63 83


In [398]:
np.mean(wer_values)

0.4836415842966014

In [399]:
np.mean(cer_values)

0.27888075993272465

In [406]:
df_clean = df.copy()

In [407]:
for i, text in enumerate(df_clean["DETECTED_TEXT"]):
    gt = df_clean["GROUND_TRUTH"][i].lower()
    gt = gt.replace(".", "").replace(",", "").replace("'", "").replace("!", "").replace("?", "").replace('"', '')
    gt = gt.replace("\\", "").replace("-", " ").replace(":", "").replace(".", "")
    remove_digits = str.maketrans('', '', digits)
    gt = gt.translate(remove_digits)
    df_clean["GROUND_TRUTH"][i] = gt

    text = text.lower()
    text = text.replace(".", "").replace(",", "").replace("'", "").replace("!", "").replace("?", "").replace('"', '')
    text = text.replace("\\", "").replace("-", " ").replace(":", "").replace(".", "")
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    df_clean["DETECTED_TEXT"][i] = text

In [408]:
df_clean.loc[(df_clean['GROUND_TRUTH'] == " ") & (df_clean['DETECTED_TEXT'] != ""), "GROUND_TRUTH"] = "nAn"

In [409]:
wer_values = []
cer_values = []
counter_0 = 0
counter_01 = 0
counter_02 = 0
counter_05 = 0
counter_worst = 0

for i in range(len(df_clean)):
    hypo = df_clean["DETECTED_TEXT"][i]
    ref = df_clean["GROUND_TRUTH"][i]
    
    if hypo == ref:
        cer_i = 0
        wer_i = 0
        counter_0 += 1
    else:
        cer_i = np.where(cer(ref, hypo) > 1, 1, cer(ref, hypo))
        wer_i = np.where(wer(ref, hypo) > 1, 1, wer(ref, hypo))
        
        if wer_i < 0.1:
            counter_01 += 1
        elif wer_i < 0.2:
            counter_02 += 1
        elif wer_i < 0.5:
            counter_05 += 1
        else:
            counter_worst += 1
        
        
    
    #print(i,"WER", wer_i)
    #print(i, "CER", cer_i)
    
    wer_values.append(wer_i)
    cer_values.append(cer_i)

In [410]:
np.mean(wer_values)

0.23970806613603604

In [411]:
np.mean(cer_values)

0.17927724818980512

In [412]:
print(counter_0, counter_01, counter_02, counter_05, counter_worst)

58 16 26 58 26
